In [1]:
from google.colab import files
uploaded = files.upload()

Saving Student Survey - Jan.xlsx to Student Survey - Jan.xlsx


In [1]:
import pandas as pd

file_path = "/content/Student Survey - Jan.xlsx"  # Adjust if needed
sheet_dict = pd.read_excel(file_path, sheet_name=None)  # Load all sheets into a dictionary

# Access individual sheets
df_affiliations = sheet_dict.get("affiliations")
df_participants = sheet_dict.get("participants")
df_responses = sheet_dict.get("responses")
df_friends = sheet_dict.get("net_0_Friends")
df_influential = sheet_dict.get("net_1_Influential")
df_feedback = sheet_dict.get("net_2_Feedback")
df_more_time = sheet_dict.get("net_3_MoreTime")
df_advice = sheet_dict.get("net_4_Advice")
df_disrespect = sheet_dict.get("net_5_Disrespect")
df_school_activity = sheet_dict.get("net_affiliation_0_SchoolActivit")

In [3]:
!pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 3.2 MB/s eta 0:00:00


In [2]:
import random
import numpy as np
import pandas as pd
import networkx as nx
from deap import base, creator, tools, algorithms
import matplotlib.pyplot as plt

#### Encoding relationships to graphs

In [3]:
# Assuming df_friends is already loaded
# Remove self-loops (where Source == Target)
df_friends = df_friends[df_friends["Source"] != df_friends["Target"]]

# Create a directed graph (DiGraph)
G = nx.DiGraph()
for _, row in df_friends.iterrows():
    G.add_edge(row["Source"], row["Target"])

# Create a dictionary to store friendships
friendship_dict = {node: list(G.neighbors(node)) for node in G.nodes()}

# Number of students and classes (predefined)
num_students = df_participants['Participant-ID'].nunique()
num_classes = 6

In [4]:
# Assuming df_influential is already loaded
# Remove self-loops (where Source == Target)
df_influential = df_influential[df_influential["Source"] != df_influential["Target"]]

# Create a directed graph (DiGraph)
G_influence = nx.DiGraph()
for _, row in df_influential.iterrows():
    G_influence.add_edge(row["Source"], row["Target"])

# Create a dictionary to store influential relationships
influence_dict = {node: list(G_influence.neighbors(node)) for node in G_influence.nodes()}

In [5]:
# Assuming df_feedback is already loaded
# Remove self-loops (where Source == Target)
df_feedback = df_feedback[df_feedback["Source"] != df_feedback["Target"]]

# Create a directed graph (DiGraph)
G_feedback = nx.DiGraph()
for _, row in df_feedback.iterrows():
    G_feedback.add_edge(row["Source"], row["Target"])

# Create a dictionary to store feedback relationships
feedback_dict = {node: list(G_feedback.neighbors(node)) for node in G_feedback.nodes()}

In [6]:
# Assuming df_more_time is already loaded
# Remove self-loops (where Source == Target)
df_more_time = df_more_time[df_more_time["Source"] != df_more_time["Target"]]

# Create a directed graph (DiGraph)
G_more_time = nx.DiGraph()
for _, row in df_more_time.iterrows():
    G_more_time.add_edge(row["Source"], row["Target"])

# Create a dictionary to store "more time" relationships
more_time_dict = {node: list(G_more_time.neighbors(node)) for node in G_more_time.nodes()}

In [7]:
# Assuming df_advice is already loaded
# Remove self-loops (where Source == Target)
df_advice = df_advice[df_advice["Source"] != df_advice["Target"]]

# Create a directed graph (DiGraph)
G_advice = nx.DiGraph()
for _, row in df_advice.iterrows():
    G_advice.add_edge(row["Source"], row["Target"])

# Create a dictionary to store advice relationships
advice_dict = {node: list(G_advice.neighbors(node)) for node in G_advice.nodes()}

In [8]:
# Assuming df_disrespect is already loaded
# Remove self-loops (where Source == Target)
df_disrespect = df_disrespect[df_disrespect["Source"] != df_disrespect["Target"]]

# Create a directed graph (DiGraph)
G_disrespect = nx.DiGraph()
for _, row in df_disrespect.iterrows():
    G_disrespect.add_edge(row["Source"], row["Target"])

# Create a dictionary to store disrespect relationships
disrespect_dict = {node: list(G_disrespect.neighbors(node)) for node in G_disrespect.nodes()}

#### CP-SAT

In [11]:
!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0


In [12]:
from ortools.sat.python import cp_model
import pandas as pd

# Assuming df_participants is already loaded with student data
student_ids = df_participants['Participant-ID'].tolist()

# Create a model instance
model = cp_model.CpModel()

# Define variables
num_students = len(student_ids)
num_classes = 6
students = student_ids
classes = range(num_classes)

# Class assignments for each student
class_assignments = {student_id: model.NewIntVar(0, num_classes - 1, f'class_{student_id}') for student_id in students}

# Define class sizes (strict class sizes)
class_capacities = [29, 29, 29, 30, 29, 29]  # Class capacities for each class
balance_threshold = 1  # Increased allowed difference between classes (+/- 3 students)

# Add "one student per class" constraint
for student_id in students:
    model.Add(class_assignments[student_id] >= 0)
    model.Add(class_assignments[student_id] < num_classes)  # Ensure the class assignment is valid

# Create strict class size constraints with a relaxed balance threshold
for c in classes:
    bool_vars = [model.NewBoolVar(f'student_{student_id}_in_class_{c}') for student_id in students]
    for student_id in students:
        model.Add(class_assignments[student_id] == c).OnlyEnforceIf(bool_vars[students.index(student_id)])
        model.Add(class_assignments[student_id] != c).OnlyEnforceIf(bool_vars[students.index(student_id)].Not())

    # Relax class size constraint further
    model.Add(sum(bool_vars) >= class_capacities[c] - balance_threshold)
    model.Add(sum(bool_vars) <= class_capacities[c] + balance_threshold)

# Solve with solution limits
solver = cp_model.CpSolver()

# Set up a solution counter to limit the number of solutions to 10 for faster feedback
max_solutions = 10
solutions = []

# Track previously found solutions to avoid duplicates
found_solutions = set()

# Set a max time for the solver (e.g., 30 seconds)
solver.parameters.max_time_in_seconds = 30

status = solver.Solve(model)

while status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    # Extract the solution as a tuple (this will ensure it's hashable for set operations)
    solution = tuple(solver.Value(class_assignments[student_id]) for student_id in students)

    # Ensure the solution is not a duplicate
    if solution not in found_solutions:
        solutions.append(solution)
        found_solutions.add(solution)

    # Stop once 10 solutions are found
    if len(solutions) >= max_solutions:
        break

    # Solve for the next solution
    status = solver.Solve(model)

# Print all the solutions
for solution in solutions:
    print(dict(zip(students, solution)))

# Print total number of solutions
print(f"Total number of allocation options generated: {len(solutions)}")

{32437: 0, 32540: 3, 32443: 5, 32514: 4, 32485: 1, 32436: 1, 32553: 4, 32463: 0, 32444: 3, 32428: 4, 32494: 0, 32484: 2, 32541: 1, 32445: 4, 32470: 3, 32395: 0, 32531: 3, 32440: 3, 32550: 0, 32511: 0, 32556: 2, 32549: 1, 32400: 2, 32505: 2, 32417: 4, 32478: 1, 32477: 2, 32474: 3, 32493: 0, 32413: 3, 32459: 1, 32534: 4, 32527: 4, 32532: 5, 32399: 1, 32409: 2, 32475: 2, 32462: 0, 32498: 4, 32458: 1, 32404: 0, 32517: 0, 32401: 5, 32504: 3, 32448: 0, 32522: 1, 32509: 1, 32397: 5, 32523: 3, 32512: 5, 32455: 4, 32489: 5, 32432: 4, 32476: 0, 32516: 5, 32467: 1, 32408: 3, 32420: 2, 32497: 2, 32515: 2, 32429: 4, 32415: 3, 32435: 3, 32405: 5, 32392: 4, 32495: 3, 32427: 5, 32421: 4, 32422: 4, 32500: 4, 32403: 3, 32557: 3, 32499: 3, 32433: 5, 32402: 5, 32491: 2, 32453: 3, 32530: 2, 32544: 1, 32508: 1, 32407: 5, 32565: 5, 32529: 5, 32457: 2, 32545: 5, 32492: 1, 32406: 4, 32479: 0, 32490: 4, 32513: 5, 32480: 4, 32546: 0, 32535: 4, 32456: 5, 32551: 5, 32446: 3, 32451: 5, 32425: 1, 32471: 0, 32528: 1,

#### GREEDY ALGO

In [9]:
import pandas as pd
import random

# Assuming df_participants is already loaded with student data
student_ids = df_participants['Participant-ID'].tolist()
gpas = df_participants['Perc_Academic'].tolist()

# Class sizes (e.g., 29 or 30 students per class)
class_capacities = [29, 29, 29, 30, 29, 29]  # Class capacities for each class
num_classes = len(class_capacities)

# Number of solutions to generate
max_solutions = 100

# Initialize to store solutions
all_solutions = []

# Function to generate one solution
def generate_solution():
    # Shuffle students randomly
    students = student_ids[:]
    random.shuffle(students)  # Shuffle students for randomness

    # Initialize class assignments and class sizes
    class_assignments = {student_id: None for student_id in student_ids}
    class_sizes = [0] * num_classes  # To track number of students assigned to each class

    # Greedy assignment: Assign students to classes
    for student_id in students:
        # Find the class with the least number of students that isn't full
        for c in range(num_classes):
            if class_sizes[c] < class_capacities[c]:
                # Assign student to class c
                class_assignments[student_id] = c
                class_sizes[c] += 1
                break

    return class_assignments

# Generate multiple solutions
while len(all_solutions) < max_solutions:
    solution = generate_solution()

    # Check if this solution is distinct
    if solution not in all_solutions:
        all_solutions.append(solution)

# Output the solutions
for idx, solution in enumerate(all_solutions):
    print(f"Solution {idx + 1}: {solution}")

# Print the total number of solutions generated
print(f"Total number of allocation options generated: {len(all_solutions)}")

Solution 1: {32437: 2, 32540: 5, 32443: 1, 32514: 1, 32485: 5, 32436: 0, 32553: 3, 32463: 2, 32444: 1, 32428: 2, 32494: 0, 32484: 3, 32541: 3, 32445: 4, 32470: 2, 32395: 1, 32531: 2, 32440: 3, 32550: 1, 32511: 0, 32556: 3, 32549: 2, 32400: 5, 32505: 2, 32417: 1, 32478: 0, 32477: 3, 32474: 1, 32493: 2, 32413: 2, 32459: 1, 32534: 0, 32527: 4, 32532: 3, 32399: 1, 32409: 4, 32475: 5, 32462: 1, 32498: 3, 32458: 4, 32404: 4, 32517: 1, 32401: 1, 32504: 0, 32448: 2, 32522: 5, 32509: 3, 32397: 3, 32523: 5, 32512: 2, 32455: 5, 32489: 5, 32432: 2, 32476: 3, 32516: 3, 32467: 3, 32408: 5, 32420: 0, 32497: 0, 32515: 3, 32429: 0, 32415: 1, 32435: 4, 32405: 0, 32392: 3, 32495: 3, 32427: 3, 32421: 1, 32422: 5, 32500: 4, 32403: 2, 32557: 0, 32499: 4, 32433: 0, 32402: 0, 32491: 0, 32453: 5, 32530: 4, 32544: 3, 32508: 5, 32407: 4, 32565: 2, 32529: 3, 32457: 0, 32545: 2, 32492: 5, 32406: 3, 32479: 2, 32490: 3, 32513: 0, 32480: 2, 32546: 1, 32535: 5, 32456: 3, 32551: 0, 32446: 4, 32451: 0, 32425: 5, 32471: 

#### GA: Fitness

In [10]:
# Define weights for each objective (adjust these based on your priorities)
weight_friendship = 1.0
weight_influence = 1.0
weight_feedback = 1.0
weight_time = 1.0
weight_advice = 1.0
weight_disrespect = 1.0

# Updated fitness functions that use weights

def calculate_friendship_density(individual):
    class_friendships = {i: [] for i in range(num_classes)}
    for student_id, class_id in individual.items():
        class_friendships[class_id].append(student_id)

    total_friendships = 0
    total_possible_friendships = 0

    for class_id, students in class_friendships.items():
        if len(students) < 2:
            continue  # Skip classes with fewer than 2 students
        for i in range(len(students)):
            for j in range(i + 1, len(students)):
                student1 = students[i]
                student2 = students[j]
                if student2 in friendship_dict.get(student1, []):  # Safe access to friendship_dict
                    total_friendships += 1
                total_possible_friendships += 1

    return total_friendships / total_possible_friendships if total_possible_friendships > 0 else 0

def calculate_influence_balance(individual):
    degree_centrality = nx.degree_centrality(G_influence)
    class_influence = {i: [] for i in range(num_classes)}
    for student_id, class_id in individual.items():
        if student_id in degree_centrality:
            class_influence[class_id].append(degree_centrality[student_id])

    total_influence = 0
    total_classes = 0
    for class_id, students in class_influence.items():
        if len(students) == 0:
            continue
        class_avg_influence = sum(students) / len(students)
        total_influence += class_avg_influence
        total_classes += 1

    return total_influence / total_classes if total_classes > 0 else 0

def calculate_feedback_exchange(individual):
    class_feedback = {i: [] for i in range(num_classes)}
    for student_id, class_id in individual.items():
        class_feedback[class_id].append(student_id)

    total_feedback = 0
    total_possible_feedback = 0
    for class_id, students in class_feedback.items():
        if len(students) < 2:
            continue
        for i in range(len(students)):
            for j in range(i + 1, len(students)):
                student1 = students[i]
                student2 = students[j]
                if student2 in feedback_dict.get(student1, []):
                    total_feedback += 1
                total_possible_feedback += 1

    return total_feedback / (total_possible_feedback if total_possible_feedback > 0 else 1)

def calculate_time_spent_together(individual):
    class_time = {i: [] for i in range(num_classes)}
    for student_id, class_id in individual.items():
        class_time[class_id].append(student_id)

    total_time = 0
    total_possible_time = 0
    for class_id, students in class_time.items():
        if len(students) < 2:
            continue
        for i in range(len(students)):
            for j in range(i + 1, len(students)):
                student1 = students[i]
                student2 = students[j]
                if student2 in more_time_dict.get(student1, []):
                    total_time += 1
                total_possible_time += 1

    return total_time / (total_possible_time if total_possible_time > 0 else 1)

def calculate_advice_exchange(individual):
    class_advice = {i: [] for i in range(num_classes)}
    for student_id, class_id in individual.items():
        class_advice[class_id].append(student_id)

    total_advice = 0
    total_possible_advice = 0
    for class_id, students in class_advice.items():
        if len(students) < 2:
            continue
        for i in range(len(students)):
            for j in range(i + 1, len(students)):
                student1 = students[i]
                student2 = students[j]
                if student2 in advice_dict.get(student1, []):
                    total_advice += 1
                total_possible_advice += 1

    return total_advice / (total_possible_advice if total_possible_advice > 0 else 1)

def calculate_disrespect(individual):
    class_disrespect = {i: [] for i in range(num_classes)}
    for student_id, class_id in individual.items():
        class_disrespect[class_id].append(student_id)

    total_disrespect = 0
    for class_id, students in class_disrespect.items():
        if len(students) < 2:
            continue
        for i in range(len(students)):
            for j in range(i + 1, len(students)):
                student1 = students[i]
                student2 = students[j]
                if student2 in disrespect_dict.get(student1, []):
                    total_disrespect += 1

    return total_disrespect

# Define the multi-objective fitness function with weighted objectives
def calculate_multi_objective_fitness(individual):
    # Calculate each objective
    friendship_density = calculate_friendship_density(individual)
    influence_balance = calculate_influence_balance(individual)
    feedback_exchange = calculate_feedback_exchange(individual)
    time_spent_together = calculate_time_spent_together(individual)
    advice_exchange = calculate_advice_exchange(individual)
    disrespect = calculate_disrespect(individual)

    # Apply weights to each objective
    weighted_friendship = friendship_density * weight_friendship
    weighted_influence = influence_balance * weight_influence
    weighted_feedback = feedback_exchange * weight_feedback
    weighted_time = time_spent_together * weight_time
    weighted_advice = advice_exchange * weight_advice
    weighted_disrespect = disrespect * weight_disrespect

    # Return the tuple of weighted fitness values
    return weighted_friendship, weighted_influence, weighted_feedback, weighted_time, weighted_advice, weighted_disrespect

GA: running

In [14]:
# GA setup
creator.create("FitnessMulti", base.Fitness, weights=(1.0, 1.0, 1.0, 1.0, 1.0, -1.0))  # Adjust weights for your problem
creator.create("Individual", dict, fitness=creator.FitnessMulti)

# Convert greedy solutions into GA-compatible individuals
def convert_to_individual(greedy_solution):
    individual = creator.Individual({student_id: greedy_solution[student_id] for student_id in student_ids})
    individual.fitness.values = calculate_multi_objective_fitness(individual)
    return individual

# Use the `all_solutions` list as the initial population for GA
population = [convert_to_individual(solution) for solution in all_solutions]

def repair(individual, class_capacities):
    """
    Fix any class overflows by moving students to underloaded classes.
    """
    num_classes = len(class_capacities)

    # Count students per class
    class_counts = {i: 0 for i in range(num_classes)}
    for class_id in individual.values():
        class_counts[class_id] += 1

    # Identify over- and under-loaded classes
    overloaded = [cid for cid in range(num_classes) if class_counts[cid] > class_capacities[cid]]
    underloaded = [cid for cid in range(num_classes) if class_counts[cid] < class_capacities[cid]]

    # Map class to students
    class_to_students = {i: [] for i in range(num_classes)}
    for sid, cid in individual.items():
        class_to_students[cid].append(sid)

    # Start repairing
    for cid in overloaded:
        while class_counts[cid] > class_capacities[cid]:
            # Pick student to move
            student_to_move = random.choice(class_to_students[cid])
            moved = False
            # Try to find a class to move to
            for target_cid in underloaded:
                if class_counts[target_cid] < class_capacities[target_cid]:
                    # Reassign student
                    individual[student_to_move] = target_cid
                    class_counts[cid] -= 1
                    class_counts[target_cid] += 1
                    class_to_students[cid].remove(student_to_move)
                    class_to_students[target_cid].append(student_to_move)
                    moved = True
                    break
            if not moved:
                break  # No room anywhere, give up

    return individual

# Mutation function (randomly change one student's class assignment)
def mutate(individual):
    student_id = random.choice(list(individual.keys()))
    individual[student_id] = random.randint(0, num_classes - 1)
    individual = repair(individual, class_capacities)
    individual.fitness.values = calculate_multi_objective_fitness(individual)
    return individual,

# Crossover function (swap assignments between two individuals)
def crossover(ind1, ind2):
    cxpoint = random.randint(1, len(ind1) - 1)
    keys = list(ind1.keys())
    for i in range(cxpoint, len(keys)):
        key = keys[i]
        ind1[key], ind2[key] = ind2[key], ind1[key]
    ind1 = repair(ind1, class_capacities)
    ind2 = repair(ind2, class_capacities)
    ind1.fitness.values = calculate_multi_objective_fitness(ind1)
    ind2.fitness.values = calculate_multi_objective_fitness(ind2)
    return ind1, ind2

# Define DEAP creators and repair here...
creator.create("FitnessMulti", base.Fitness, weights=(1.0, 1.0, 1.0, 1.0, 1.0, -1.0))
creator.create("Individual", dict, fitness=creator.FitnessMulti)

# Setup toolbox
toolbox = base.Toolbox()
toolbox.register("individual", tools.initIterate, creator.Individual, convert_to_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", crossover)
toolbox.register("mutate", mutate)
toolbox.register("select", tools.selNSGA2)
toolbox.register("evaluate", calculate_multi_objective_fitness)

# Run GA
population_size = 1000
num_generations = 50
algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=num_generations,
                    stats=None, halloffame=None, verbose=True)

# Validate best solution
best_individual = tools.selBest(population, 1)[0]
print(f"Best solution (class assignments): {best_individual}")

/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals
0  	0     
1  	73    
2  	71    
3  	74    
4  	70    
5  	84    
6  	78    
7  	70    
8  	72    
9  	80    
10 	76    
11 	76    
12 	82    
13 	76    
14 	76    
15 	78    
16 	74    
17 	74    
18 	71    
19 	76    
20 	73    
21 	78    
22 	69    
23 	71    
24 	66    
25 	79    
26 	77    
27 	71    
28 	74    
29 	82    
30 	86    
31 	87    
32 	73    
33 	73    
34 	75    
35 	75    
36 	77    
37 	78    
38 	68    
39 	74    
40 	70    
41 	76    
42 	80    
43 	83    
44 	74    
45 	71    
46 	78    
47 	74    
48 	74    
49 	76    
50 	72    
Best solution (class assignments): {32437: 0, 32540: 4, 32443: 0, 32514: 3, 32485: 2, 32436: 1, 32553: 1, 32463: 3, 32444: 0, 32428: 3, 32494: 2, 32484: 3, 32541: 5, 32445: 2, 32470: 2, 32395: 1, 32531: 2, 32440: 4, 32550: 4, 32511: 4, 32556: 2, 32549: 0, 32400: 0, 32505: 3, 32417: 3, 32478: 5, 32477: 2, 32474: 5, 32493: 5, 32413: 0, 32459: 4, 32534: 1, 32527: 4, 32532: 3, 32399: 2, 32409: 5, 32475: 3, 32462: 4, 32498: 3, 32

In [17]:
import random
from deap import base, creator, tools, algorithms

# === Step 1: Define DEAP creators ===
creator.create("FitnessMulti", base.Fitness, weights=(1.0, 1.0, 1.0, 1.0, 1.0, -1.0))  # Adjust based on your needs
creator.create("Individual", dict, fitness=creator.FitnessMulti)

# === Step 2: Repair function for hard constraints (class size) ===
def repair(individual, class_capacities):
    num_classes = len(class_capacities)
    class_counts = {i: 0 for i in range(num_classes)}
    for class_id in individual.values():
        class_counts[class_id] += 1

    overloaded = [cid for cid in range(num_classes) if class_counts[cid] > class_capacities[cid]]
    underloaded = [cid for cid in range(num_classes) if class_counts[cid] < class_capacities[cid]]

    class_to_students = {i: [] for i in range(num_classes)}
    for sid, cid in individual.items():
        class_to_students[cid].append(sid)

    for cid in overloaded:
        while class_counts[cid] > class_capacities[cid]:
            student_to_move = random.choice(class_to_students[cid])
            moved = False
            for target_cid in underloaded:
                if class_counts[target_cid] < class_capacities[target_cid]:
                    individual[student_to_move] = target_cid
                    class_counts[cid] -= 1
                    class_counts[target_cid] += 1
                    class_to_students[cid].remove(student_to_move)
                    class_to_students[target_cid].append(student_to_move)
                    moved = True
                    break
            if not moved:
                break
    return individual

# === Step 3: Random individual generator ===
def random_allocation():
    individual = {}
    shuffled_students = student_ids[:]
    random.shuffle(shuffled_students)

    class_sizes = [0] * num_classes
    for sid in shuffled_students:
        # Assign student to the first class with space
        for cid in range(num_classes):
            if class_sizes[cid] < class_capacities[cid]:
                individual[sid] = cid
                class_sizes[cid] += 1
                break
    return creator.Individual(repair(individual, class_capacities))

# === Step 4: Genetic operators ===

def mutate(individual):
    student_id = random.choice(list(individual.keys()))
    individual[student_id] = random.randint(0, num_classes - 1)
    individual = repair(individual, class_capacities)
    individual.fitness.values = calculate_multi_objective_fitness(individual)
    return individual,

def crossover(ind1, ind2):
    cxpoint = random.randint(1, len(ind1) - 1)
    keys = list(ind1.keys())
    for i in range(cxpoint, len(keys)):
        key = keys[i]
        ind1[key], ind2[key] = ind2[key], ind1[key]
    ind1 = repair(ind1, class_capacities)
    ind2 = repair(ind2, class_capacities)
    ind1.fitness.values = calculate_multi_objective_fitness(ind1)
    ind2.fitness.values = calculate_multi_objective_fitness(ind2)
    return ind1, ind2

# === Step 5: Setup toolbox ===
toolbox = base.Toolbox()
toolbox.register("individual", random_allocation)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", crossover)
toolbox.register("mutate", mutate)
toolbox.register("select", tools.selNSGA2)
toolbox.register("evaluate", calculate_multi_objective_fitness)

# === Step 6: Run the GA ===
population_size = 100
num_generations = 50

# Initialize population
population = toolbox.population(n=population_size)

# Evaluate initial fitness
for ind in population:
    ind.fitness.values = calculate_multi_objective_fitness(ind)

# Run algorithm
algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=num_generations,
                    stats=None, halloffame=None, verbose=True)

# === Step 7: Output best result ===
best_individual = tools.selBest(population, 1)[0]
print(f"\nBest class allocation (GA only): {best_individual}")

gen	nevals
0  	0     
1  	90    
2  	72    
3  	76    
4  	73    
5  	77    
6  	80    
7  	79    
8  	77    
9  	71    
10 	75    
11 	83    
12 	71    
13 	82    
14 	88    
15 	72    
16 	84    
17 	63    
18 	74    
19 	78    
20 	85    
21 	76    
22 	83    
23 	74    
24 	65    
25 	67    
26 	69    
27 	71    
28 	88    
29 	75    
30 	82    
31 	74    
32 	72    
33 	84    
34 	73    
35 	72    
36 	81    
37 	85    
38 	65    
39 	76    
40 	84    
41 	72    
42 	76    
43 	81    
44 	59    
45 	69    
46 	60    
47 	75    
48 	69    
49 	68    
50 	79    

Best class allocation (GA only): {32425: 0, 32437: 1, 32452: 0, 32556: 3, 32414: 3, 32508: 0, 32532: 2, 32516: 1, 32411: 1, 32417: 4, 32399: 2, 32538: 5, 32540: 2, 32422: 0, 32392: 2, 32459: 3, 32519: 1, 32420: 2, 32460: 0, 32520: 2, 32488: 4, 32495: 0, 32458: 5, 32499: 4, 32562: 5, 32493: 1, 32531: 4, 32544: 2, 32505: 0, 32415: 2, 32416: 3, 32484: 0, 32521: 0, 32391: 0, 32397: 3, 32461: 4, 32436: 2, 32490: 3, 32560: 1, 324

In [15]:
def check_hard_constraints(individual, class_capacities, student_ids):
    """
    Validates that a GA-generated class allocation satisfies hard constraints:
    - No class exceeds its capacity
    - All students are assigned exactly once

    Parameters:
    - individual: dict of {student_id: class_id}
    - class_capacities: list of max class sizes
    - student_ids: list of all valid student IDs

    Returns:
    - (is_valid: bool, error_messages: list of str)
    """

    num_classes = len(class_capacities)
    class_sizes = [0] * num_classes
    error_messages = []

    # Check if all students are assigned exactly once
    if set(individual.keys()) != set(student_ids):
        missing = set(student_ids) - set(individual.keys())
        extra = set(individual.keys()) - set(student_ids)
        if missing:
            error_messages.append(f"Missing assignments for student IDs: {missing}")
        if extra:
            error_messages.append(f"Extra assignments for unknown student IDs: {extra}")
        return False, error_messages

    # Check class sizes
    for student_id, class_id in individual.items():
        if class_id < 0 or class_id >= num_classes:
            error_messages.append(f"Invalid class ID {class_id} for student {student_id}")
            continue
        class_sizes[class_id] += 1

    for i, size in enumerate(class_sizes):
        if size > class_capacities[i]:
            error_messages.append(f"Class {i} is over capacity: {size} > {class_capacities[i]}")

    is_valid = len(error_messages) == 0
    return is_valid, error_messages

In [18]:
is_valid, errors = check_hard_constraints(best_individual, class_capacities, student_ids)

if is_valid:
    print("✅ Best GA solution satisfies all hard constraints.")
else:
    print("❌ GA solution violates hard constraints:")
    for err in errors:
        print(" -", err)

✅ Best GA solution satisfies all hard constraints.


Evaluation

In [ ]:
# Get the best individual (solution with the highest fitness)
best_individual = tools.selBest(population, 1)[0]

# Evaluate the fitness of the best individual
fitness_score = calculate_multi_objective_fitness(best_individual)

# Print the fitness values for each objective
print("Fitness values for the best individual (class assignments):")
print(f"Friendship density: {fitness_score[0]}")
print(f"Influence balance: {fitness_score[1]}")
print(f"Feedback exchange: {fitness_score[2]}")
print(f"Time spent together: {fitness_score[3]}")
print(f"Advice exchange: {fitness_score[4]}")
print(f"Disrespect: {fitness_score[5]}")

# You can also print the class assignments of the best individual
print(f"Best individual (class assignments): {best_individual}")

In [17]:
# Extract the Pareto front (solutions that are not dominated)
pareto_front = tools.sortNondominated(population, len(population), first_front_only=True)[0]

# Print the Pareto front (solutions that balance the objectives)
print("Pareto front solutions (class assignments and fitness scores):")
for ind in pareto_front:
    print(f"Class assignments: {ind}, Fitness: {ind.fitness.values}")

Pareto front solutions (class assignments and fitness scores):
Class assignments: {32437: 5, 32540: 2, 32443: 0, 32514: 0, 32485: 4, 32436: 2, 32553: 3, 32463: 4, 32444: 4, 32428: 2, 32494: 0, 32484: 0, 32541: 1, 32445: 0, 32470: 3, 32395: 0, 32531: 5, 32440: 2, 32550: 4, 32511: 1, 32556: 4, 32549: 0, 32400: 2, 32505: 1, 32417: 1, 32478: 3, 32477: 3, 32474: 5, 32493: 4, 32413: 0, 32459: 4, 32534: 3, 32527: 1, 32532: 5, 32399: 1, 32409: 5, 32475: 4, 32462: 5, 32498: 1, 32458: 3, 32404: 0, 32517: 0, 32401: 4, 32504: 3, 32448: 4, 32522: 5, 32509: 2, 32397: 1, 32523: 2, 32512: 4, 32455: 5, 32489: 5, 32432: 4, 32476: 5, 32516: 2, 32467: 4, 32408: 3, 32420: 3, 32497: 5, 32515: 1, 32429: 4, 32415: 0, 32435: 4, 32405: 5, 32392: 4, 32495: 5, 32427: 1, 32421: 1, 32422: 4, 32500: 3, 32403: 5, 32557: 0, 32499: 0, 32433: 1, 32402: 3, 32491: 0, 32453: 3, 32530: 0, 32544: 3, 32508: 2, 32407: 3, 32565: 5, 32529: 0, 32457: 1, 32545: 3, 32492: 0, 32406: 1, 32479: 5, 32490: 3, 32513: 2, 32480: 3, 32546: 